In [ ]:
# 1. Proje Kurulumu ve Güncelleme (Colab venv ile)
import os
import sys

IN_COLAB = False
try:
    from google.colab import drive  # type: ignore
    IN_COLAB = True
except ImportError:
    pass

if IN_COLAB:
    repo_url = "https://github.com/onndd/newproje.git"
    project_dir = "/content/newproje"
    if not os.path.exists(project_dir):
        !git clone {repo_url} {project_dir}
    %cd {project_dir}
    !git pull

    venv_dir = "/content/jetx-venv"
    if not os.path.exists(venv_dir):
        !python -m venv {venv_dir}
    pip_cmd = f"{venv_dir}/bin/pip"
    python_cmd = f"{venv_dir}/bin/python"
    !{pip_cmd} install --upgrade pip

    # Çekirdek paketler
    !{pip_cmd} install numpy==1.26.2 pandas==2.1.4 matplotlib==3.8.2 seaborn==0.13.0 joblib==1.3.2 scikit-learn==1.5.2

    # Model çekirdekleri (dep çekmeden)
    !{pip_cmd} install --no-deps catboost==1.2.8 lightgbm==4.6.0 hmmlearn==0.3.2

    # Diğerleri
    !{pip_cmd} install streamlit==1.29.0 optuna==3.5.0 stable-baselines3==2.2.1 shimmy==1.3.0 gymnasium==0.29.1 "gymnasium[box2d]" tensorflow==2.16.1

    sys.path.insert(0, os.path.join(venv_dir, 'lib', 'python3.12', 'site-packages'))
else:
    project_dir = os.getcwd()
    sys.path.append(project_dir)


In [ ]:
# 2. Veri Yükleme ve 3'lü Ayrım (3-Way Split)
import sqlite3
import pandas as pd
import numpy as np
from jetx_project.data_loader import load_data
from jetx_project.config import DB_PATH

print("Veri yükleniyor...")
try:
    df = load_data(DB_PATH)
    print(f"Toplam {len(df)} veri yüklendi.")
    
    # --- 3-WAY SPLIT STRATEJİSİ ---
    # 1. Base Train (%70): Temel modelleri eğitmek için.
    # 2. Meta Train (%15): Temel modellerin tahminlerini alıp Meta-Learner'ı eğitmek için.
    # 3. Test (%15): Final simülasyon ve değerlendirme için (Hiçbir model burayı görmez).
    
    total_len = len(df)
    base_train_end = int(total_len * 0.70)
    meta_train_end = int(total_len * 0.85)
    
    print(f"--- Veri Ayrımı ---")
    print(f"Base Train: 0 - {base_train_end} ({base_train_end} örnek)")
    print(f"Meta Train: {base_train_end} - {meta_train_end} ({meta_train_end - base_train_end} örnek)")
    print(f"Test (Sim): {meta_train_end} - {total_len} ({total_len - meta_train_end} örnek)")
    
except Exception as e:
    print(f"Hata: {e}")
    print("Lütfen jetx.db dosyasının proje klasöründe olduğundan emin olun.")

In [ ]:
# 2.5. Anomaly Detector (The Shield) Eğitimi
from jetx_project.model_anomaly import train_anomaly_detector, save_anomaly_detector

if 'df' in locals() and len(df) > 500:
    print("Anomaly Detector (The Shield) eğitiliyor...")
    # Base Train verisiyle eğit
    train_values_anomaly = df['value'].values[:base_train_end]
    anomaly_model = train_anomaly_detector(train_values_anomaly)
    save_anomaly_detector(anomaly_model, output_dir='models')
    print("Anomaly Detector tamamlandı.")
else:
    print("Yeterli veri yok.")


In [ ]:
# 3. Psikolojik Modeller (HMM) Eğitimi
print("Psikolojik Modeller (HMM) eğitiliyor...")
from jetx_project.model_hmm import train_categorical_hmm, predict_categorical_hmm_states_causal, save_hmm_model

if 'df' in locals() and len(df) > 0:
    # Sadece Base Train verisiyle eğit
    train_values = df['value'].values[:base_train_end]
    
    # HMM Eğitimi (Categorical)
    hmm_model, hmm_map, hmm_bins = train_categorical_hmm(train_values)
    save_hmm_model(hmm_model, hmm_map, bins=hmm_bins, output_dir='models')
    
    # Tüm veri seti için durum tahmini (Feature olarak kullanılacak)
    # CAUSAL PREDICTION: Geleceği görmeden, kayan pencere ile tahmin et.
    all_values = df['value'].values
    hmm_states_mapped = predict_categorical_hmm_states_causal(hmm_model, all_values, hmm_map, bins=hmm_bins, window_size=200)
    
    print("HMM Durumları (Causal) tüm veri için belirlendi.")
    print(f"Durum Dağılımı: {np.bincount(hmm_states_mapped)}")
else:
    print("Veri yok, HMM eğitilemedi.")

In [ ]:
# 4. OPTUNA İLE HİPERPARAMETRE OPTİMİZASYONU (Base Train Üzerinde)
from jetx_project.optimization import optimize_catboost, optimize_lightgbm
from jetx_project.model_a import prepare_model_a_data

if 'df' in locals() and len(df) > 500:
    print("Optimizasyon için veri hazırlanıyor...")
    # Tüm veriyi hazırla ama sadece Base Train kısmını kullan
    X, y_p15, y_p3, y_x = prepare_model_a_data(df['value'].values, hmm_states_mapped)
    
    # Base Train Maskesi
    # X indexleri orijinal df indexleriyle uyumlu olmalı (prepare_model_a_data koruyor mu?)
    # prepare_model_a_data içinde valid_mask var, indexleri resetlemiyor sanırım.
    # Güvenli olmak için iloc kullanalım, ancak X'in uzunluğu len(df) - 1 (shift yüzünden).
    
    # X'in satır sayısı len(df)-1 civarı. 
    # Base Train End indexine kadar olanları alacağız.
    
    # Basitçe: İlk %70'lik dilim.
    limit_idx = base_train_end
    
    # X bir DataFrame, indexleri kontrol edelim
    X_train_opt = X[X.index < limit_idx]
    # y numpy array, maske ile filtrele
    mask_opt = (X.index < limit_idx)
    y_p15_train_opt = y_p15[mask_opt]
    
    print(f"Optimizasyon Veri Seti: {len(X_train_opt)} satır")
    
    print("--- CatBoost Optimizasyonu Başlıyor (GPU) ---")
    best_params_catboost = optimize_catboost(X_train_opt, y_p15_train_opt, n_trials=20)
    print("Bulunan En İyi Parametreler:", best_params_catboost)
else:
    print("Yeterli veri yok.")

In [ ]:
# 5. Model A (CatBoost) Eğitimi (Base Train)
from jetx_project.model_a import train_model_a, save_models
from catboost import CatBoostClassifier

if 'df' in locals() and len(df) > 500:
    print("Model A (CatBoost) eğitiliyor...")
    
    # Sadece Base Train verisini kullan
    limit_idx = base_train_end
    mask_train = (X.index < limit_idx)
    
    X_train_base = X[mask_train]
    y_p15_base = y_p15[mask_train]
    y_p3_base = y_p3[mask_train]
    y_x_base = y_x[mask_train]
    
    # train_model_a kendi içinde %15 validasyon ayırır (Base Train'in %15'i)
    modelA_p15, modelA_p3, modelA_x = train_model_a(X_train_base, y_p15_base, y_p3_base, y_x_base)
    save_models(modelA_p15, modelA_p3, modelA_x, output_dir='models')
    print("Model A tamamlandı.")
else:
    print("Yeterli veri yok.")

In [ ]:
# 6. Model B (k-NN) Eğitimi (Base Train)
from jetx_project.model_b import build_memory, train_model_b, save_memory

if 'df' in locals() and len(df) > 500:
    print("Model B (Hafıza) oluşturuluyor...")
    
    # Sadece Base Train verisiyle hafıza oluştur
    train_values_b = df['value'].values[:base_train_end]
    patterns, targets = build_memory(train_values_b)
    
    nbrs, pca = train_model_b(patterns)
    
    save_memory(nbrs, pca, patterns, targets, output_dir='models')
    print("Model B tamamlandı.")
else:
    print("Yeterli veri yok.")

In [ ]:
# 7. Model C (LSTM) Eğitimi (Base Train)
from jetx_project.model_lstm import train_model_lstm, save_lstm_models

if 'df' in locals() and len(df) > 500:
    print("Model C (LSTM) eğitiliyor...")
    
    # Sadece Base Train verisi
    train_values_c = df['value'].values[:base_train_end]
    
    modelC_p15, modelC_p3, scaler_lstm = train_model_lstm(train_values_c)
    save_lstm_models(modelC_p15, modelC_p3, scaler_lstm, output_dir='models')
    print("Model C tamamlandı.")
else:
    print("Yeterli veri yok.")

In [ ]:
# 8. Model D (LightGBM) Eğitimi (Base Train)
from jetx_project.model_lightgbm import train_model_lightgbm, save_lightgbm_models

if 'df' in locals() and len(df) > 500:
    print("Model D (LightGBM) eğitiliyor...")
    modelD_p15, modelD_p3 = train_model_lightgbm(X_train_base, y_p15_base, y_p3_base)
    save_lightgbm_models(modelD_p15, modelD_p3, output_dir='models')
    print("Model D tamamlandı.")
else:
    print("Yeterli veri yok.")

In [ ]:
# 9. Model E (MLP) Eğitimi (Base Train)
from jetx_project.model_mlp import train_model_mlp, save_mlp_models

if 'df' in locals() and len(df) > 500:
    print("Model E (MLP) eğitiliyor...")
    modelE_p15, modelE_p3, mlp_cols = train_model_mlp(X_train_base, y_p15_base, y_p3_base)
    save_mlp_models(modelE_p15, modelE_p3, mlp_cols, output_dir='models')
    print("Model E tamamlandı.")
else:
    print("Yeterli veri yok.")

In [ ]:
# 9.1. Model F (Transformer - The Attention) Eğitimi (Base Train)
from jetx_project.model_transformer import train_model_transformer, save_transformer_models

if 'df' in locals() and len(df) > 500:
    print("Model F (Transformer) eğitiliyor...")
    # Base Train verisi
    train_values_f = df['value'].values[:base_train_end]
    
    model_transformer, scaler_transformer = train_model_transformer(train_values_f)
    save_transformer_models(model_transformer, scaler_transformer, output_dir='models')
    print("Model F (Transformer) tamamlandı.")
else:
    print("Yeterli veri yok.")


In [ ]:
# 9.5. ENSEMBLE STACKING (Meta-Learner) Eğitimi (Meta Train)
from jetx_project.ensemble import prepare_meta_features, train_meta_learner, save_meta_learner, predict_meta
from jetx_project.model_lstm import create_sequences

if 'df' in locals() and len(df) > 2000:
    print("--- Ensemble Stacking (Meta-Learner) Eğitiliyor ---")
    
    meta_start_idx = base_train_end
    meta_end_idx = meta_train_end
    
    print(f"Meta Train Aralığı: {meta_start_idx} - {meta_end_idx}")
    
    # 1. Meta Train Seti İçin Tahminleri Topla
    
    # Model A (CatBoost)
    X_meta, _, _, _ = prepare_model_a_data(df['value'].values, hmm_states_mapped, start_index=meta_start_idx)
    X_meta = X_meta[X_meta.index < meta_end_idx]
    
    # Hedefler
    y_meta_true_15 = (df['value'].values[meta_start_idx+1 : meta_end_idx+1] >= 1.5).astype(int)
    
    # Uzunluk Kontrolü
    min_len = min(len(X_meta), len(y_meta_true_15))
    X_meta = X_meta.iloc[:min_len]
    y_meta_true_15 = y_meta_true_15[:min_len]
    
    print(f"Meta Train Örnek Sayısı: {min_len}")
    
    preds_a_meta = modelA_p15.predict_proba(X_meta)[:, 1]
    
    # Model B (k-NN)
    preds_b_meta = []
    from jetx_project.model_b import create_pattern_vector, predict_model_b
    pca_obj = locals().get('pca', None) 
    
    for idx in X_meta.index:
        pat = create_pattern_vector(df['value'].values, idx)
        if pat is not None:
            p15, _, _ = predict_model_b(nbrs, pca_obj, targets, pat)
            preds_b_meta.append(p15)
        else:
            preds_b_meta.append(0.5)
    preds_b_meta = np.array(preds_b_meta)
            
    # Model C (LSTM)
    seq_len = 200
    lstm_start_offset = X_meta.index[0] - seq_len
    lstm_end_offset = X_meta.index[-1]
    
    meta_values_extended = df['value'].values[lstm_start_offset : lstm_end_offset + 1]
    meta_values_scaled = scaler_lstm.transform(meta_values_extended.reshape(-1, 1))
    
    X_lstm_meta, _, _, _ = create_sequences(meta_values_scaled, seq_len)
    preds_c_meta = modelC_p15.predict(X_lstm_meta).flatten()
    
    # Model D (LightGBM)
    preds_d_meta = modelD_p15.predict_proba(X_meta)[:, 1]
    
    # Model E (MLP)
    X_meta_mlp = X_meta[mlp_cols]
    preds_e_meta = modelE_p15.predict_proba(X_meta_mlp)[:, 1]

    # Model F (Transformer)
    meta_values_scaled_trans = scaler_transformer.transform(meta_values_extended.reshape(-1, 1))
    X_trans_meta, _, _, _ = create_sequences(meta_values_scaled_trans, seq_len)
    preds_f_meta = model_transformer.predict(X_trans_meta)
    preds_f_meta_p15 = preds_f_meta[0].flatten()
    
    # HMM States
    hmm_meta = hmm_states_mapped[X_meta.index]
    
    # 2. Meta-Features Hazırla
    final_len = min(len(preds_a_meta), len(preds_b_meta), len(preds_c_meta), len(preds_d_meta), len(preds_e_meta), len(preds_f_meta_p15))
    
    meta_values_raw = df['value'].values[meta_start_idx : meta_end_idx]
    meta_values_raw = meta_values_raw[:final_len]
    
    meta_X_train = prepare_meta_features(
        preds_a_meta[:final_len], 
        preds_b_meta[:final_len], 
        preds_c_meta[:final_len], 
        preds_d_meta[:final_len], 
        preds_e_meta[:final_len], 
        hmm_meta[:final_len],
        values=meta_values_raw,
        preds_transformer=preds_f_meta_p15[:final_len]
    )
    y_meta_train = y_meta_true_15[:final_len]
    
    # 3. Eğit
    meta_model, meta_scaler = train_meta_learner(meta_X_train, y_meta_train)
    save_meta_learner(meta_model, meta_scaler, output_dir='models')
    
    print("Meta-Learner eğitildi ve kaydedildi.")
else:
    print("Yeterli veri yok.")


In [ ]:
# 9.8. RL Agent (The Strategist) Eğitimi
from jetx_project.model_rl import train_rl_agent, save_rl_agent

if 'meta_model' in locals():
    print("RL Agent (The Strategist) eğitiliyor...")
    
    # Meta-Learner tahminlerini alalım
    meta_probs = predict_meta(meta_model, meta_scaler, meta_X_train)
    
    # DataFrame oluştur
    rl_train_df = pd.DataFrame({
        'value': meta_values_raw, # Gerçek değerler
        'prob_1.5': meta_probs,
        'hmm_state': hmm_meta[:final_len] # HMM durumları
    })
    
    rl_agent = train_rl_agent(rl_train_df)
    save_rl_agent(rl_agent, output_dir='models')
    print("RL Agent tamamlandı.")
else:
    print("Meta-Learner hazır değil.")


In [ ]:
# 10. BÜYÜK FİNAL: Simülasyon (Test Seti)
import matplotlib.pyplot as plt
from jetx_project.simulation import run_simulation
from jetx_project.evaluation import detailed_evaluation
from jetx_project.ensemble import predict_meta_safe
from jetx_project.model_rl import predict_action
from jetx_project.model_anomaly import check_anomaly

if 'df' in locals() and len(df) > 2000:
    print("--- BÜYÜK FİNAL: Simülasyon Başlıyor (Test Seti) ---")
    
    test_start_idx = meta_train_end
    test_df = df.iloc[test_start_idx:].copy()
    
    print(f"Test Seti Başlangıç: {test_start_idx}, Uzunluk: {len(test_df)}")
    
    # --- Tahminleri Topla ---
    print("Modeller tahmin ediyor...")
    X_test_a, _, _, _ = prepare_model_a_data(df['value'].values, hmm_states_mapped, start_index=test_start_idx)
    sim_df = test_df.iloc[:-1].copy()
    min_len = min(len(X_test_a), len(sim_df))
    X_test_a = X_test_a.iloc[:min_len]
    sim_df = sim_df.iloc[:min_len]
    
    # Model A
    sim_df['prob_A_1.5'] = modelA_p15.predict_proba(X_test_a)[:, 1]
    sim_df['prob_A_3.0'] = modelA_p3.predict_proba(X_test_a)[:, 1]
    
    # Model C (LSTM)
    seq_len = 200
    lstm_start = test_start_idx - seq_len
    test_values_extended = df['value'].values[lstm_start:]
    test_values_scaled = scaler_lstm.transform(test_values_extended.reshape(-1, 1))
    X_lstm, _, _, _ = create_sequences(test_values_scaled, seq_len)
    sim_df['prob_C_1.5'] = modelC_p15.predict(X_lstm).flatten()[:min_len]
    
    # Model D (LightGBM)
    sim_df['prob_D_1.5'] = modelD_p15.predict_proba(X_test_a)[:, 1]
    
    # Model E (MLP)
    X_test_mlp = X_test_a[mlp_cols]
    sim_df['prob_E_1.5'] = modelE_p15.predict_proba(X_test_mlp)[:, 1]
    
    # Model F (Transformer)
    test_values_scaled_trans = scaler_transformer.transform(test_values_extended.reshape(-1, 1))
    X_trans, _, _, _ = create_sequences(test_values_scaled_trans, seq_len)
    preds_f = model_transformer.predict(X_trans)
    sim_df['prob_F_1.5'] = preds_f[0].flatten()[:min_len]

    # Model B (k-NN)
    preds_b_sim = []
    pca_obj = locals().get('pca', None)
    for idx in sim_df.index:
        pat = create_pattern_vector(df['value'].values, idx)
        if pat is not None:
            p15, _, _ = predict_model_b(nbrs, pca_obj, targets, pat)
            preds_b_sim.append(p15)
        else:
            preds_b_sim.append(0.5)
    sim_df['prob_B_1.5'] = preds_b_sim
    
    # --- ENSEMBLE (Meta-Learner) ---
    print("Ensemble (Meta-Learner) tahmin ediyor...")
    hmm_sim = hmm_states_mapped[test_start_idx : test_start_idx + len(sim_df)]
    
    meta_X_sim = prepare_meta_features(
        sim_df['prob_A_1.5'], 
        sim_df['prob_B_1.5'], 
        sim_df['prob_C_1.5'], 
        sim_df['prob_D_1.5'], 
        sim_df['prob_E_1.5'], 
        hmm_sim,
        values=sim_df['value'].values,
        preds_transformer=sim_df['prob_F_1.5']
    )
    
    sim_df['prob_Ensemble_1.5'] = predict_meta(meta_model, meta_scaler, meta_X_sim)
    
    # --- RL AGENT SİMÜLASYONU ---
    print("\n>>> RL Agent (The Strategist) Simülasyonu <<<")
    
    balance = 1000.0
    initial_balance = 1000.0
    history_balance = [balance]
    
    # Anomaly Detector için son 300 veriyi tutmamız lazım
    window_data = list(df['value'].values[test_start_idx-300 : test_start_idx])
    
    for i in range(len(sim_df)):
        row = sim_df.iloc[i]
        true_val = row['value']
        
        # 1. Anomaly Check (The Shield)
        current_window = window_data[-300:]
        score, _ = check_anomaly(anomaly_model, current_window)
        
        if score == -1:
            # Anomaly Detected! Pass.
            action = 0 # Pass
        else:
            # 2. RL Agent Decision (The Strategist)
            recent_trend = np.mean(window_data[-10:])
            
            action = predict_action(
                rl_agent, 
                row['prob_Ensemble_1.5'], 
                0.0, 
                hmm_sim[i], 
                balance, 
                recent_trend,
                initial_balance
            )
            
        # Execute Action
        bet = 10.0
        if action == 1: # Bet 1.5x
            if true_val >= 1.5:
                balance += bet * 0.5
            else:
                balance -= bet
        elif action == 2: # Bet 2.0x
            if true_val >= 2.0:
                balance += bet * 1.0
            else:
                balance -= bet
        
        history_balance.append(balance)
        window_data.append(true_val)
        
    # Plot Balance
    plt.figure(figsize=(12, 6))
    plt.plot(history_balance)
    plt.title("RL Agent Balance History")
    plt.xlabel("Games")
    plt.ylabel("Balance")
    plt.show()
    
    print(f"Final Balance: {balance:.2f}")

else:
    print("Simülasyon için yeterli veri yok.")


In [ ]:
# 11. Modelleri Yedekle (Google Drive & Local)
import os
import datetime
import shutil
from google.colab import drive, files

print("--- Modeller Yedekleniyor ---")

# 1. Zip Oluştur
print("Modeller sıkıştırılıyor (models.zip)...")
!zip -r models.zip models/

# 2. Google Drive Yedekleme
try:
    print("Google Drive bağlanıyor...")
    drive.mount('/content/drive')
    
    # Zaman damgalı klasör adı
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M")
    backup_folder_name = f"JetX_Backup_{timestamp}"
    backup_path = f"/content/drive/MyDrive/{backup_folder_name}"
    
    if not os.path.exists(backup_path):
        os.makedirs(backup_path)
        print(f"Yedekleme klasörü oluşturuldu: {backup_path}")
        
    # Kopyala
    shutil.copy("models.zip", f"{backup_path}/models.zip")
    print(f"✅ BAŞARILI: Modeller Drive'a yedeklendi -> {backup_path}/models.zip")
    
except Exception as e:
    print(f"⚠️ UYARI: Google Drive yedeklemesi başarısız oldu: {e}")
    print("Yerel indirme deneniyor...")

# 3. Yerel İndirme (Her durumda sunulur)
try:
    files.download('models.zip')
    print("✅ İndirme başlatıldı.")
except Exception as e:
    print(f"İndirme hatası: {e}")
